In [1]:
"""
Parse value sets from i2b2 metadataxml. Tested only for a specific request that was fulfilled in early 2018.
Scrubs dates and excludes excessively long numbers, which crept into our data but were PHI.
by Jeff Klann, PhD 2-2018

Intended to be used with this SQL statement output to ClemLabRequest:
select distinct isnull(nc.c_name,m.c_name),m.c_basecode,c_loinc,m.C_METADATAXML from i2b2metadata  m left outer join RPDR_Labtests l on l.c_basecode=m.C_BASECODE
 left outer join PCORI_Dev..NCBO_LOINC_2014AB nc on nc.C_BASECODE='LOINC:'+c_loinc
 where m.c_fullname like '\i2b2metadata\LabTests%' and c_loinc is not null and c_loinc!='' and m.C_SYNONYM_CD='N' and nc.C_SYNONYM_CD='N' and l.c_synonym_cd='N'
 
"""

"\nIntended to be used with this SQL statement output to ClemLabRequest:\nselect distinct isnull(nc.c_name,m.c_name),m.c_basecode,c_loinc,m.C_METADATAXML from i2b2metadata  m left outer join RPDR_Labtests l on l.c_basecode=m.C_BASECODE\n left outer join PCORI_Dev..NCBO_LOINC_2014AB nc on nc.C_BASECODE='LOINC:'+c_loinc\n where m.c_fullname like '\\i2b2metadata\\LabTests%' and c_loinc is not null and c_loinc!='' and m.C_SYNONYM_CD='N' and nc.C_SYNONYM_CD='N' and l.c_synonym_cd='N'\n \n"

In [2]:
# conda install -c conda-forge xmltodict
import xmltodict
import csv
from tqdm import tqdm,tnrange, tqdm_notebook
import collections as c
import inspect, re 
import dateutil.parser as dateparse


In [9]:
cnt_e = 0 
cnt_o = 0
out=open('/Users/jeffklann/Dropbox (Partners HealthCare)/HMS/Projects/Clem/ClemOut.csv', 'w', newline='')
outwriter=csv.writer(out, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
# Write header row
outval=['c_name','i_loinc','value_count','num_dt']
outval=outval+['v'+str(e) for e in range(70)]
outwriter.writerow(outval)
with open('/Users/jeffklann/Dropbox (Partners HealthCare)/HMS/Projects/Clem/ClemLabRequest.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(spamreader)
    for row in tqdm(spamreader):
        if row[3]!="":
            #print(row[2][21:])
            doc = xmltodict.parse(row[3][21:])
            #print(doc.keys())
            #print(doc['ValueMetadata'].keys())
            if(doc['ValueMetadata']['DataType']==None): continue
            if(doc['ValueMetadata']['DataType']!='Enum'): continue
            if('Val' not in doc['ValueMetadata']['EnumValues'].keys()): continue
            cnt_e=cnt_e+1
            enum=doc['ValueMetadata']['EnumValues']['Val']
            vset=[]
            if(enum==None): continue
            try:
                for e in enum:
                    if isinstance(e,c.OrderedDict):
                        if (e['@description']!='') and (e['@description']!=e['#text']):
                            v = e['@description']+":"+e['#text']
                        else:
                            v = e['#text']
                        vset.append(v)
            except:
                print("ERR in: "+str(enum)+", signal"+str(row[3]))
                continue
            if len(vset)>0:
                cnt_num=0
                cnt_o=cnt_o+1
                # Detect long numbers and scrub dates
                # Note: slower approach than converting vset to a single string bc we want to modify original
                for i,v in enumerate(vset):
                    m=re.findall("[0-9\-/]+",v)
                    for mr in m:
                        # Count num of long strings
                        if len (mr)>5 and mr[-2:-1]!='-': cnt_num=cnt_num+1 
                        # Remove dates
                        if mr.count('/')==2 or mr.count('-')==2 and len(mr)>5: 
                            try:
                                dateparse.parse(mr)
                                vset[i]=v.replace(mr,'mmddyy')
                            except Exception as ex:
                                None   
            
                if cnt_num<=5 and len(vset)<=70: # Exclude rows with >5 long numbers or >70 total values
                    outval=[row[0],row[2],len(vset),cnt_num]+vset
                    outwriter.writerow(outval)
    out.close()
                    
print(str(cnt_e)+','+str(cnt_o))

41357it [00:18, 2294.96it/s]

8909,5626
